# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Deep-Learning" data-toc-modified-id="Deep-Learning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Deep Learning</a></div><div class="lev2 toc-item"><a href="#Assignment-1" data-toc-modified-id="Assignment-1-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Assignment 1</a></div><div class="lev2 toc-item"><a href="#Problem-1" data-toc-modified-id="Problem-1-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Problem 1</a></div><div class="lev2 toc-item"><a href="#Problem-2" data-toc-modified-id="Problem-2-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Problem 2</a></div><div class="lev2 toc-item"><a href="#Problem-3" data-toc-modified-id="Problem-3-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Problem 3</a></div><div class="lev2 toc-item"><a href="#Problem-4" data-toc-modified-id="Problem-4-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Problem 4</a></div><div class="lev2 toc-item"><a href="#Problem-5" data-toc-modified-id="Problem-5-16"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Problem 5</a></div>

Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [55]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from random import randint
import time
from sets import Set as set
#from sets import Set as set

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

ImportError: cannot import name 'Set'

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [3]:
url = 'http://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None
data_root = '/Users/gundeepsingh/Documents/Notebook/'

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(dest_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', dest_filename)
  else:
    raise Exception(
      'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Attempting to download: notMNIST_large.tar.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified /Users/gundeepsingh/Documents/Notebook/notMNIST_large.tar.gz
Attempting to download: notMNIST_small.tar.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified /Users/gundeepsingh/Documents/Notebook/notMNIST_small.tar.gz


Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [5]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

Extracting data for /Users/gundeepsingh/Documents/Notebook/notMNIST_large. This may take a while. Please wait.
['/Users/gundeepsingh/Documents/Notebook/notMNIST_large/A', '/Users/gundeepsingh/Documents/Notebook/notMNIST_large/B', '/Users/gundeepsingh/Documents/Notebook/notMNIST_large/C', '/Users/gundeepsingh/Documents/Notebook/notMNIST_large/D', '/Users/gundeepsingh/Documents/Notebook/notMNIST_large/E', '/Users/gundeepsingh/Documents/Notebook/notMNIST_large/F', '/Users/gundeepsingh/Documents/Notebook/notMNIST_large/G', '/Users/gundeepsingh/Documents/Notebook/notMNIST_large/H', '/Users/gundeepsingh/Documents/Notebook/notMNIST_large/I', '/Users/gundeepsingh/Documents/Notebook/notMNIST_large/J']
Extracting data for /Users/gundeepsingh/Documents/Notebook/notMNIST_small. This may take a while. Please wait.
['/Users/gundeepsingh/Documents/Notebook/notMNIST_small/A', '/Users/gundeepsingh/Documents/Notebook/notMNIST_small/B', '/Users/gundeepsingh/Documents/Notebook/notMNIST_small/C', '/Users/g

---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. 

A few images might not be readable, we'll just skip them.

In [ ]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `train_size` as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [7]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (200000, 28, 28) (200000,)
Validation: (10000, 28, 28) (10000,)
Testing: (10000, 28, 28) (10000,)


Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [8]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

---

Finally, let's save the data for later reuse:

In [9]:
pickle_file = os.path.join(data_root, 'notMNIST.pickle')

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [1]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

NameError: name 'os' is not defined

---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [61]:
train_dataset.flags.writeable=False
test_dataset.flags.writeable=False
dup_table = {}

def maybe_print_images(image1, image2):
    rand = randint(0,99)
    if(rand == 50):
        print('print images -- -- -- ')
        # img1 = Image.fromarray(image1)
        # img2 = Image.fromarray(image2)
        plt.imshow(image1)
        plt.show
        plt.figure()
        plt.imshow(image2)
        plt.show
        print('justfinished printing...')
    

def check_overlap(dataset1, dataset2):
    df_1 = 0
    df_2 = 0
    for idx,image in enumerate(dataset1[:10000]):
        h = hash(image.tobytes())
        if h  in dup_table and (dataset1[dup_table[h]].tobytes() == image.tobytes()):
            # print('duplicate found: ' + str(idx) + ' and ' + str(dup_table[h]))
            df_1 = df_1 + 1
        dup_table[h] = idx
    for idx,image in enumerate(dataset2):
        h = hash(image.tobytes())
        if h  in dup_table and (dataset1[dup_table[h]].tobytes() == image.tobytes()):
            # print('test set index ' + str(idx) + ' matches index ' + str(dup_table[h]) + 'in training set')
            maybe_print_images(test_dataset[idx], train_dataset[dup_table[h]])
            df_2 = df_2 + 1
    return df_1,df_2

def check_overlaps_2(ds1, ds2):
    ds1.flags.writeable = False
    ds2.flags.writeable = False
    start = time.clock()
    set1 = set(hash(image.tobytes()) for image in ds1)
    set2 = set(hash(image.tobytes()) for image in ds2)
    all_overlaps = set.intersection(set1, set2)
    return all_overlaps,time.clock()-start

# df1, df2 = check_overlap(train_dataset, test_dataset)
# print('duplicates found in training data = ' + str(df1))
# print('duplicates in training and test data = ' + str(df2))
all_overlaps,time_diff = check_overlaps_2(train_dataset, test_dataset)
print('nubmer of overlaps between train and test data = ' + str(len(all_overlaps)) + '| time take = ' + str(time_diff))
all_overlaps,time_diff = check_overlaps_2(train_dataset, valid_dataset)
print('nubmer of overlaps between train and validation data = ' + str(len(all_overlaps)) + '| time take = ' + str(time_diff))
all_overlaps,time_diff = check_overlaps_2(valid_dataset, test_dataset)
print('nubmer of overlaps between validation and test data = ' + str(len(all_overlaps)) + '| time take = ' + str(time_diff))
    

nubmer of overlaps between train and test data = 1153| time take = 0.6672770000000128
nubmer of overlaps between train and validation data = 953| time take = 0.7539049999999179
nubmer of overlaps between validation and test data = 55| time take = 0.06662500000004457


---
Problem 6

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [111]:
shp = train_dataset.shape
print(shp)
flat_50 = train_dataset[:50].reshape(50, -1)
print(flat_50.shape)

(200000, 28, 28)
(50, 784)


In [122]:
logres = LogisticRegression()


def train_model(tds, tl, vds, vl, count):
    flat_tds = tds[:count].reshape(count,-1)
    flat_vds = vds[:count].reshape(count,-1)
    X = flat_tds
    Y = tl[:count]
    X2 = flat_vds
    Y2 = vl[:count]
    print('will try to fit X and Y, shape of X = ' + str(X.shape) + ' and shape of Y = ' + str(Y.shape))
    logres.fit(X,Y)
    prediction = Y2 == logres.predict(X2)
    correctPredictions = np.sum(prediction)
    print('correct predictions = ' + str(correctPredictions) + ' total count = ' + str(count))
    accuracy = correctPredictions/count
    print('score = ' + str(logres.score(X2, Y2)))
    print('accuracy = ' + str(accuracy))

train_model(train_dataset, train_labels, valid_dataset, valid_labels, 50)
train_model(train_dataset, train_labels, valid_dataset, valid_labels, 100)
train_model(train_dataset, train_labels, valid_dataset, valid_labels, 1000)
train_model(train_dataset, train_labels, valid_dataset, valid_labels, 5000)

will try to fit X and Y, shape of X = (50, 784) and shape of Y = (50,)
correct predictions = 24 total count = 50
score = 0.48
accuracy = 0.48
will try to fit X and Y, shape of X = (100, 784) and shape of Y = (100,)
correct predictions = 67 total count = 100
score = 0.67
accuracy = 0.67
will try to fit X and Y, shape of X = (1000, 784) and shape of Y = (1000,)
correct predictions = 736 total count = 1000
score = 0.736
accuracy = 0.736
will try to fit X and Y, shape of X = (5000, 784) and shape of Y = (5000,)
correct predictions = 3849 total count = 5000
score = 0.7698
accuracy = 0.7698


In [102]:
ar = np.arange(60).reshape(-1,3,4)
print(str(ar) + '\n')
br = ar[1,]
print(br)
print('size = ' + str(ar.shape[0]))
print(ar.size)
cr = ar.reshape(ar.shape[0], -1)
print(cr)

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]

 [[24 25 26 27]
  [28 29 30 31]
  [32 33 34 35]]

 [[36 37 38 39]
  [40 41 42 43]
  [44 45 46 47]]

 [[48 49 50 51]
  [52 53 54 55]
  [56 57 58 59]]]

[[12 13 14 15]
 [16 17 18 19]
 [20 21 22 23]]
size = 5
60
[[ 0  1  2  3  4  5  6  7  8  9 10 11]
 [12 13 14 15 16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31 32 33 34 35]
 [36 37 38 39 40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55 56 57 58 59]]


In [120]:
x = y = z = 1
print(x)
print(y)
print(z)

1
1
1
